# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

H. Beuther  ->  H. Beuther  |  ['H. Beuther']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
N. Wang  ->  N. Wang  |  ['N. Wang']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']


E. Bañados  ->  E. Bañados  |  ['E. Bañados']
S. Belladitta  ->  S. Belladitta  |  ['S. Belladitta']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
M. Schirmer  ->  M. Schirmer  |  ['M. Schirmer']


Arxiv has 116 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2512.00147


extracting tarball to tmp_2512.00147...

 done.


H. Beuther  ->  H. Beuther  |  ['H. Beuther']


...

extracting tarball to tmp_2512.01606...

 done.


M. Schirmer  ->  M. Schirmer  |  ['M. Schirmer']


/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure imf_dndm_modeltestcompl_corrected_mag_cut_0.2_whole0.022_comb_scaled_bin0.01
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:137: LatexWarning: Recovered figure imf_dndm_modeltestcompl_corrected_mag_cut_0.2_whole0.022_comb_scaled_bin0.01 as tmp_2512.01606/./imf_dndm_modeltestcompl_corrected_mag_cut_0.2_whole0.022_comb_scaled_bin0.01.png
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure imf_dndm_VISTAtestcompl_corrected_mag_cut_0.2_whole0.022_comb_scaled_bin0.01
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.25/x64/lib/pytho

Found 57 bibliographic references in tmp_2512.01606/aa56578-25.bbl.
Issues with the citations
syntax error in line 30: '}' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.00147-b31b1b.svg)](https://arxiv.org/abs/2512.00147) | **The ALMA Survey of 70 μm Dark High-mass Clumps in Early Stages (ASHES). XIII. Core Mass Function, Lifetime, and Growth of Prestellar Cores**  |
|| K. Morii, et al. -- incl., <mark>H. Beuther</mark> |
|*Appeared on*| *2025-12-02*|
|*Comments*| *Accepted for Publication in ApJ., 18 pages, 10 figures*|
|**Abstract**|            The core mass function (CMF) of prestellar cores is essential for understanding the initial conditions of star and cluster formation. However, the universality of the CMF and its relationship to the initial mass function (IMF) remain unclear. We study the CMF in the earliest stage of high-mass star formation using 461 prestellar core candidates and 254 protostellar cores as a part of the ALMA Survey of 70 \mu m Dark High-mass Clumps in Early Stages (ASHES). We find that prestellar core candidates tend to have lower masses than protostellar cores. We also find that the lifetime of prestellar cores is several times longer than the freefall time, although it approaches the freefall time as the core mass increases. The CMF, including both protostellar and prestellar cores, has a power-law slope of $-2.05\pm0.04$, shallower than Salpeter's IMF slope of -2.35. Conversely, the CMF of gravitationally bound, prestellar cores has a steeper slope ($-2.32\pm0.30$), indistinguishable from Salpeter's slope. This finding is consistent with observations in both low-mass star-forming regions and high-mass protoclusters, implying a universal core formation mechanism. The protostellar CMF with a larger maximum core mass can be reproduced by the prestellar CMF when an external gas infall is considered. The inferred mass infall rate is higher than the Bondi-Hoyle-Lyttleton accretion rate and follows a shallower mass dependence (smaller power-law index), more consistent with the tidal-lobe accretion. This may contribute to the evolution of CMFs seen in later stages.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.00157-b31b1b.svg)](https://arxiv.org/abs/2512.00157) | **Astrometric view of companions in the inner dust cavities of protoplanetary disks**  |
|| M. Vioque, et al. -- incl., <mark>M. Benisty</mark> |
|*Appeared on*| *2025-12-02*|
|*Comments*| *Accepted for publication in A&A. Table A.1 is available in electronic form (Sect. 7)*|
|**Abstract**|            Protoplanetary disks with inner dust cavities (often referred to as "transition disks") are potential signposts of planet formation. We use Gaia astrometry to search for planetary and stellar companions in a sample of 98 transition disks, assessing the occurrence rate of such companions and their potential influence on cavity formation. For the 98 Young Stellar Objects (YSOs), we compute Gaia proper motion anomalies which, together with the RUWE, identify companions with mass ratios $q \gtrsim 0.01$ at $\sim$0.1-30 au. We assess the impact of disk gravity, accretion, disk-scattered light, dippers, starspots, jets, and outflows on the measured proper motion anomalies, concluding that astrometric techniques such as the one of this work can be robustly applied to YSOs. Significant proper motion anomalies are found in 31 transition disks (32% of the sample), indicative of companions. We recover 85% of known companions within our sensitivity range. We model the semi-major axis and mass required for a companion to reproduce the observed astrometric signals. Most inferred companions have $M > 30$ M$\rm{_{J}}$, placing many within or near the stellar mass regime. Seven sources host companions compatible with a planetary mass ($M < 13$ M$\rm{_{J}}$, HD 100453, J04343128+1722201, J16102955-3922144, MHO6, MP Mus, PDS 70, and Sz 76). For the non-detections, we provide the companion masses and semi-major axes that can be excluded in future searches. About half (53%) of detected companions cannot be reconciled with having carved the observed dust cavities. We find that transition disks host as many companions within our sensitivity range as do randomly sampled groups of YSOs and main-sequence stars. If dust cavities are shaped by companions, such companions must reside at larger orbital separations than those of the companions detected here, and we predict them to be of planetary mass. [abridged]         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.01075-b31b1b.svg)](https://arxiv.org/abs/2512.01075) | **Euclid Structural-Thermal-Optical Performance**  |
|| E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-12-02*|
|*Comments*| *21 pages, 15 figures, submitted to A&A*|
|**Abstract**|            The Euclid system performance is defined in terms of image quality metrics tuned to the weak gravitational lensing (WL) cosmological probe. WL induces stringent requirements on the shape and stability of the VIS instrument system point spread function (PSF). The PSF is affected by error contributions from the telescope, the focal plane and image motion, and is controlled by a global error budget with error allocations to each contributor. Aims. During spacecraft development, we verified through a structural-thermal-optical performance (STOP) analysis that the built and verified telescope with its spacecraft interface meets the in-orbit steady-state and transient image quality requirements. Methods. For the purposes of the STOP analysis, a detailed finite-element mathematical model was set up and a standard set of test cases, both steady-state and transient, was defined, comprising combinations of worst-case boundary conditions. Results. The STOP analysis addressed the interaction of all spacecraft components in transmitting temperature-induced loads that lead to optical train deformation. The results of the prelaunch analysis demonstrated that temperature-induced optical perturbations will be well below the allowable limits for all permitted observing conditions. During the first year in orbit, we used the STOP analysis predictions to help interpret the measured performance as a function of environmental variables. Unpredicted disturbances were discovered and unexpected sensitivities were revealed. In-orbit temperature variations are small (<300 mK) and so are their effects on the telescope structure, but they are detected in the time histories of the image quality metrics and are a non-negligible factor in the PSF stability budget demanded by the WL science. Taking everything into account, our analysis confirms the excellent overall performance of the telescope.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.01489-b31b1b.svg)](https://arxiv.org/abs/2512.01489) | **Euclid: The first statistical census of dusty and massive objects in the ERO/Perseus field**  |
|| G. Girardi, et al. -- incl., <mark>E. Bañados</mark>, <mark>S. Belladitta</mark>, <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-12-02*|
|*Comments*| *23 pages, 8 figures. Accepted for publication in A\&A*|
|**Abstract**|            Our comprehension of the history of star formation at $z>3$ relies on rest-frame UV observations, yet this selection misses the most dusty and massive sources, yielding an incomplete census at early times. Infrared facilities such as Spitzer and the James Webb Space Telescope have revealed a hidden population at $z=3$-$6$ with extreme red colours, named HIEROs (HST-to-IRAC extremely red objects), identified by the criterion $H_{\mathrm{E}}-\mathrm{ch2}>2.25$. Recently, Euclid Early Release Observations (ERO) have made it possible to further study such objects by comparing Euclid data with ancillary Spitzer/IRAC imaging. We investigate a $232$ arcmin$^2$ area in the Perseus field using VIS and NISP photometry, complemented by the four Spitzer channels and ground-based MegaCam bands ($u$, $g$, $r$, ${\rm H}\alpha$, $i$, $z$). Applying the colour cut yields $121$ HIEROs; after removing globular clusters, brown dwarfs, and unreliable cases through visual inspection of multiband cutouts, we obtain a final sample of $42$ robust HIEROs. Photometric redshifts and physical properties are estimated with the SED-fitting code Bagpipes. From the resulting $z_{\mathrm{phot}}$ and $M_*$ values, we compute the galaxy stellar mass function at $3.5<z<5.5$. Even after excluding possible AGN hosts or systems where the stellar mass may be overestimated, the high-mass end remains comparable to previous determinations, suggesting the true abundance could be higher. These results highlight the importance of further study of this obscured population to assess its role in the cosmic star-formation rate density and its consistency with galaxy-formation models, demonstrating Euclid's capability to advance our understanding of dust-hidden star formation across early epochs.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.01606-b31b1b.svg)](https://arxiv.org/abs/2512.01606) | **Substellar Initial Mass Function of Trumpler 14**  |
|| T. Rom, et al. -- incl., <mark>M. Schirmer</mark> |
|*Appeared on*| *2025-12-02*|
|*Comments*| **|
|**Abstract**|            Young, massive stellar clusters offer a prime setting to explore brown dwarf (BD) formation under high densities and intense UV radiation. Trumpler 14 (Tr 14), a 1 Myr-old cluster located at a distance of 2.4 kpc, and particularly rich in O-type stars, is an ideal target for such a study. Our goal is to measure the initial mass function (IMF) in the young massive, high UV flux cluster. We present the deepest study to date of the IMF in Tr 14 based on GeMS/GSAOI imaging. We construct the IMF using both the Besancon Galactic model and an observational control field from VISTA for background correction. Completeness was assessed using artificial-star tests and applied to the IMF derivation. We estimate the IMF down to the 20% completeness limit found at 0.01 MSun. Using the control field-based IMF as our primary result, we find a slope of alpha=0.14+-0.19 for masses between 0.01-0.2 MSun, and alpha=1.72+-0.04 for 0.2-4.5 MSun. The low-mass slope is largely influenced by the incompleteness-affected lowest bin; excluding it brings our results into agreement with those from other young clusters. The resulting median for the star-to-BD ratio in the 0.03-1 MSun mass range is 4.0, with a 95% confidence interval of 2.8-5.8. Our analysis reveals that Tr 14 hosts a relatively flat substellar IMF, but this is strongly influenced by the lowest-mass bin, which may be affected by incompleteness. When that bin is excluded, the IMF becomes consistent with those of other regions. The star-to-BD ratio falls within the usually observed 3-6 range, indicating that brown dwarfs with masses above 0.03 MSun form with similar efficiency across environments. However, the relative lack of objects below this threshold suggests that the presence of an environment with both high stellar density and FUV flux may play a role in shaping the IMF by suppressing the formation of BDs at masses < 0.03 MSun.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.00871-b31b1b.svg)](https://arxiv.org/abs/2512.00871) | **Possible quasi-periodic optical oscillations of ZTF blazars**  |
|| <mark>N. Wang</mark>, et al. |
|*Appeared on*| *2025-12-02*|
|*Comments*| *25 pages, 4 tables, 7 figures, accepted for publication in JHEAp*|
|**Abstract**|            Based on the Zwicky Transient Facility (ZTF), we selected 10 blazars as our sample sources. Among these, we found four blazars (J 0923.5+4125, J 1221.3+3010, J 1503.5+4759, and J 1652.7+4024) showing possible indications of quasi periodic oscillations (QPOs) modulation. We conducted a detailed analysis of their optical light curves (g- and r-bands) over the past five years using the root mean square (RMS)-Flux relation, flux distribution, and QPO detection methods to investigate their variability characteristics. A linear RMS-Flux relation is present in both bands, and their flux distributions follow a log-normal form. This suggests that optical variability may arise from multiplicative, nonlinear processes across different timescales and flux states. Further QPO analysis using the weighted wavelet Z-transform (WWZ), Lomb-Scargle periodogram (LSP), and autoregressive integrated moving average (ARIMA) methods identified candidate periodic signals in four blazars. J 0923.5+4125 (period $\sim$ 205 days) and J 1221.3+3010 ($\sim$ 630 days) show local significances of $\sim 3 \sigma$, whereas J 1503.5+4759 ($\sim$ 38.5 days) and J 1652.7+4024 ($\sim$ 48 days) reach $\sim 4 \sigma$. After accounting for the look-elsewhere effect, the global significances for J 1503.5+4759 in the g- and r-bands are $\sim 2.7 \sigma$, while for J 1652.7+4024 they are approximately $\sim 2.5 \sigma$ in both bands. These two blazars warrant further monitoring and investigation.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2512.00147/./CMF_all_class.png', 'tmp_2512.00147/./CMF_pre_prebound.png', 'tmp_2512.00147/./Recoverable_fraction_all.png', 'tmp_2512.00147/./Recoverable_flux_34cl.png']
copying  tmp_2512.00147/./CMF_all_class.png to _build/html/
copying  tmp_2512.00147/./CMF_pre_prebound.png to _build/html/
copying  tmp_2512.00147/./Recoverable_fraction_all.png to _build/html/
copying  tmp_2512.00147/./Recoverable_flux_34cl.png to _build/html/
exported in  _build/html/2512.00147.md
    + _build/html/tmp_2512.00147/./CMF_all_class.png
    + _build/html/tmp_2512.00147/./CMF_pre_prebound.png
    + _build/html/tmp_2512.00147/./Recoverable_fraction_all.png
    + _build/html/tmp_2512.00147/./Recoverable_flux_34cl.png
found figures ['tmp_2512.00157/./Plots/TDs_after_referee/AA_Tau_scatter.png', 'tmp_2512.00157/./Plots/TDs_after_referee/AA_Tau_density.png', 'tmp_2512.00157/./Plots/TDs_after_referee/AK_Sco_scatter.png', 'tmp_2512.00157/./Plots/TDs_after_referee/AK_Sco_density.png', 'tmp_2512

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\red}[1]{\textcolor{red}{#1}}$
$\newcommand{\kms}{\mbox{km s^{-1}}}$
$\newcommand{\x}{\mbox{\times}}$
$\newcommand{\Msun}{\mbox{M_{\odot}}}$
$\newcommand{\Lsun}{\mbox{L_{\odot}}}$</div>



<div id="title">

# The ALMA Survey of 70 \textmu m Dark High-mass Clumps in Early Stages (ASHES).\ XIII. Core Mass Function, Lifetime, and Growth of Prestellar Cores

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2512.00147-b31b1b.svg)](https://arxiv.org/abs/2512.00147)<mark>Appeared on: 2025-12-02</mark> -  _Accepted for Publication in ApJ., 18 pages, 10 figures_

</div>
<div id="authors">

K. Morii, et al. -- incl., <mark>H. Beuther</mark>

</div>
<div id="abstract">

**Abstract:** The core mass function (CMF) of prestellar cores is essential for understanding the initial conditions of star and cluster formation. However, the universality of the CMF and its relationship to the initial mass function (IMF) remain unclear.We study the CMF in the earliest stage of high-mass star formation using 461 prestellar core candidates and 254 protostellar cores as a part of the ALMA Survey of 70 \textmu m Dark High-mass Clumps in Early Stages (ASHES).We find that prestellar core candidates tend to have lower masses than protostellar cores.We also find that the lifetime of prestellar cores is several times longer than the freefall time, although it approaches the freefall time as the core mass increases.The CMF, including both protostellar and prestellar cores, has a power-law slope of -2.05 $\pm$ 0.04, shallower than Salpeter's IMF slope of -2.35.Conversely, the CMF of gravitationally bound, prestellar cores has a steeper slope (-2.32 $\pm$ 0.30), indistinguishable from Salpeter's slope.This finding is consistent with observations in both low-mass star-forming regions and high-mass protoclusters, implying a universal core formation mechanism.The protostellar CMF with a larger maximum core mass can be reproduced by the prestellar CMF when an external gas infall is considered.The inferred mass infall rate is higher than the Bondi-Hoyle-Lyttleton accretion rate and follows a shallower mass dependence (smaller power-law index), more consistent with the tidal-lobe accretion.This may contribute to the evolution of CMFs seen in later stages.

</div>

<div id="div_fig1">

<img src="tmp_2512.00147/./CMF_all_class.png" alt="Fig9" width="100%"/>

**Figure 9. -** CMF of (left) the full sample, (middle) the selected sample, and (right) the selected sample corrected by the recoverable flux. The three different colors represent the evolutionary stages of cores: prestellar core candidates (light blue), warm core (green), and outflow core (orange).
    Each bin in the histogram represents the combined frequency from several stages, stacked to show the relative contribution of each dataset to the total count per bin. The worst 90\% completeness (1.2 $M_\odot$) is plotted in the middle and right panels. The results of power-law fit above 1.2 $M_\odot$ are also overplotted.  (*fig:cmf-all*)

</div>
<div id="div_fig2">

<img src="tmp_2512.00147/./CMF_pre_prebound.png" alt="Fig10" width="100%"/>

**Figure 10. -** Core mass distributions of (a-c) prestellar core candidates and (d-f) gravitationally bound prestellar cores. Three columns show different plots in the form of (a, d)  differential CMFs, (b,e) cumulative CMFs, and (c, f) cumulative percentage of cores with $M_{\rm core}>M$, respectively. The best-fitting power-law index is plotted in each panel. The number of cores above 1.2 $M_\odot$ in each sample is shown.  (*fig:cmf_prebound*)

</div>
<div id="div_fig3">

<img src="tmp_2512.00147/./Recoverable_fraction_all.png" alt="Fig1.1" width="50%"/><img src="tmp_2512.00147/./Recoverable_flux_34cl.png" alt="Fig1.2" width="50%"/>

**Figure 1. -** Completeness analysis of synthetic cores embedded in the continuum image. Regions with completeness levels below 90\% at 1.2 $M_\odot$ are highlighted as colors in (a). In (b), the clumps with completeness levels below 90\% at 1.2 $M_\odot$ are excluded. The lines are colored in for the mass range with the completeness level higher than 50\%, which are used for flux correction.  (*fig:CMF_completeness*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2512.00147"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\textbf{}}[1]{#1}}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$</div>



<div id="title">

# Astrometric view of companions in the inner dust cavities of protoplanetary disks

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2512.00157-b31b1b.svg)](https://arxiv.org/abs/2512.00157)<mark>Appeared on: 2025-12-02</mark> -  _Accepted for publication in A&A. Table A.1 is available in electronic form (Sect. 7)_

</div>
<div id="authors">

M. Vioque, et al. -- incl., <mark>M. Benisty</mark>

</div>
<div id="abstract">

**Abstract:** Protoplanetary disks with inner dust cavities (often referred to as "transition disks") are potential signposts of planet formation. However, few companions have been identified within these cavities, and the role of companions in shaping them remains unclear. We use _Gaia_ astrometry to search for planetary and stellar companions in a sample of **98** transition disks, assessing the occurrence rate of such companions and their potential influence on cavity formation. For the **98** Young Stellar Objects (YSOs) with inner dust cavities, we compute _Gaia_ proper motion anomalies **which, together with the RUWE,** identify companions with mass ratios $q \gtrsim 0.01$ at $\sim$ 0.1--30 au. We assess the impact of disk gravity, accretion, disk-scattered light, dippers, **starspots** , jets, and outflows on the measured proper motion anomalies, concluding that these effects are unlikely to affect our analyses and that astrometric techniques such as the one of this work can be robustly applied to YSOs. Significant **proper motion** anomalies are found in 31 transition disks (32 \% of the sample), indicative of companions. We recover 85 \% of known companions within our sensitivity range. Assuming **that the astrometry of each system** is dominated by a single companion, we model the semi-major axis and mass required to reproduce the observed astrometric signals. Most inferred companions have $M > 30$ M $\rm{_{J}}$ , placing many within or near the stellar mass regime. Seven sources host companions compatible with a planetary mass ( $M < 13$ M $\rm{_{J}}$ , HD 100453, J04343128+1722201, J16102955-3922144, MHO6, MP Mus, PDS 70, and Sz 76). For the non-detections, we provide the companion masses and semi-major axes that can be excluded in future searches. About half (53 \% ) of detected companions cannot be reconciled with having carved the observed dust cavities. We have gathered evidence for the presence of companions in a large sample of transition disks. However, we find that the population of transition disks cannot be fully described as a circumbinary population. Transition disks host as many companions within our sensitivity range as do randomly sampled groups of YSOs and main-sequence stars. If dust cavities are shaped by companions, such companions must reside at larger orbital separations than those of the companions detected here, and we predict them to be of planetary mass.

</div>

<div id="div_fig1">

<img src="tmp_2512.00157/./Plots/TDs_after_referee/AA_Tau_scatter.png" alt="Fig15.1" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/AA_Tau_density.png" alt="Fig15.2" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/AK_Sco_scatter.png" alt="Fig15.3" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/AK_Sco_density.png" alt="Fig15.4" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/CIDA_9_A_scatter.png" alt="Fig15.5" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/CIDA_9_A_density.png" alt="Fig15.6" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/CQ_Tau_scatter.png" alt="Fig15.7" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/CQ_Tau_density.png" alt="Fig15.8" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/CS_Cha_scatter.png" alt="Fig15.9" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/CS_Cha_density.png" alt="Fig15.10" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/GW_Ori_scatter.png" alt="Fig15.11" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/GW_Ori_density.png" alt="Fig15.12" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/HD_100453_scatter.png" alt="Fig15.13" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/HD_100453_density.png" alt="Fig15.14" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/HD_142527_scatter.png" alt="Fig15.15" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/HD_142527_density.png" alt="Fig15.16" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/HP_Cha_scatter.png" alt="Fig15.17" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/HP_Cha_density.png" alt="Fig15.18" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/IRAS_04125_2902_scatter.png" alt="Fig15.19" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/IRAS_04125_2902_density.png" alt="Fig15.20" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/IRAS_04158_2805_scatter.png" alt="Fig15.21" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/IRAS_04158_2805_density.png" alt="Fig15.22" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/IRAS_16072_2057_scatter.png" alt="Fig15.23" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/IRAS_16072_2057_density.png" alt="Fig15.24" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/J04343128_1722201_scatter.png" alt="Fig15.25" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/J04343128_1722201_density.png" alt="Fig15.26" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/J16102955_3922144_scatter.png" alt="Fig15.27" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/J16102955_3922144_density.png" alt="Fig15.28" width="3%"/>

**Figure 15. -** Orbital separation and companion mass that would produce the observed _Gaia_ astrometry for the 31 sources with a significant proper motion anomaly ($|\Delta \mu|/\sigma_{|\Delta \mu|} \geq 3$). GG Tau is not shown here because its companion could not be modelled. (*fig:all_detect*)

</div>
<div id="div_fig2">

<img src="tmp_2512.00157/./Plots/TDs_after_referee/AB_Aur_density.png" alt="Fig18.1" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/AS_205_S_density.png" alt="Fig18.2" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/AS_209_density.png" alt="Fig18.3" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/AT_Pyx_density.png" alt="Fig18.4" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/BP_Tau_density.png" alt="Fig18.5" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/CIDA_1_density.png" alt="Fig18.6" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/DM_Tau_density.png" alt="Fig18.7" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/DoAr_44_density.png" alt="Fig18.8" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/EX_Lup_density.png" alt="Fig18.9" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/GM_Aur_density.png" alt="Fig18.10" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/HD_100546_density.png" alt="Fig18.11" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/HD_135344_B_density.png" alt="Fig18.12" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/HD_139614_density.png" alt="Fig18.13" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/HD_142666_density.png" alt="Fig18.14" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/HD_143006_density.png" alt="Fig18.15" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/HD_163296_density.png" alt="Fig18.16" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/HD_169142_density.png" alt="Fig18.17" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/HD_290764_density.png" alt="Fig18.18" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/HD_34282_density.png" alt="Fig18.19" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/HD_34700_density.png" alt="Fig18.20" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/HD_97048_density.png" alt="Fig18.21" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/IP_Tau_density.png" alt="Fig18.22" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/IRAS_16201_2410_density.png" alt="Fig18.23" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/IRS_48_density.png" alt="Fig18.24" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/ISO_Oph_2_density.png" alt="Fig18.25" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/J04124068_2438157_density.png" alt="Fig18.26" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/J04360131_1726120_density.png" alt="Fig18.27" width="3%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/J05080709_2427123_density.png" alt="Fig18.28" width="3%"/>

**Figure 18. -** Parameter space of orbital separations and companion masses that would produce either a significant _Gaia_ proper motion anomaly or a high RUWE value, for sources without a significant proper motion anomaly ($|\Delta \mu|/\sigma_{|\Delta \mu|} < 3$). These figures indicate the companion separation–mass parameter space in which a companion dominating the astrometric signal can be excluded. (*fig:all_non_detect*)

</div>
<div id="div_fig3">

<img src="tmp_2512.00157/./Plots/TDs_after_referee/LkCa_15_density.png" alt="Fig7.1" width="50%"/><img src="tmp_2512.00157/./Plots/TDs_after_referee/WISPIT_2_density.png" alt="Fig7.2" width="50%"/>

**Figure 7. -** Parameter space of orbital separations and companion masses that would produce either a significant _Gaia_ proper motion anomaly or a **higher-than-observed** RUWE value in LkCa 15 **and WISPIT 2**, smoothed with a kernel density estimate (contours indicate the 20, 50, and 80\% levels of the normalised area). These regions can be discarded as hosting a companion **dominating the astrometry**(Sect. \ref{s_nondetections}, Eqs. \ref{eq_6}, \ref{eq_7}).** For WISPIT 2, the confirmed (at $\sim$57 au, \citealp{2025ApJ...990L...8V**) and proposed (at $\sim$15 au, \citealp{2025ApJ...990L...9C}) planetary mass companions are indicated}. Similar plots for all other **65****systems with** non-detections are shown in Fig. \ref{fig:all_non_detect}. (*fig_LkCa15*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2512.00157"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\orcid}[1]{\orcidlink{#1}}$</div>



<div id="title">

# $\Euclid$: Structural-Thermal-Optical Performance

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2512.01075-b31b1b.svg)](https://arxiv.org/abs/2512.01075)<mark>Appeared on: 2025-12-02</mark> -  _21 pages, 15 figures, submitted to A&A_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** The $\Euclid$ system performance is defined in terms of image quality metrics tuned to the weak gravitational lensing cosmological probe. The weak lensing measurement induces stringent requirements on the shape and stability of the VIS instrument system $\ac{PSF}$ . The $\ac{PSF}$ is affected by error contributions from the telescope, the focal plane and image motion, and is controlled by a global error budget with error allocations to each contributor. During spacecraft development, we verified through a structural-thermal-optical performance (STOP) analysis that the built and verified telescope with its spacecraft interface meets the in-orbit steady-state and transient image quality requirements, under temperature-induced loads, in all permitted spacecraft attitudes after all permitted attitude transitions. In the first year in orbit, we compared the expected with the actual performance. For the purposes of the STOP analysis, a detailed finite-element mathematical model was set up and a standard set of test cases, both steady-state and transient, was defined, comprising combinations of worst-case boundary conditions. Iterations of the analysis were performed in conjunction with the major reviews of the spacecraft verification cycle. After launch, the model is used in sensitivity analyses using realistic boundary conditions. The STOP analysis addressed the interaction of all spacecraft components in transmitting temperature-induced loads that lead to optical train deformation. The results of the prelaunch analysis demonstrated that temperature-induced optical perturbations will be well below the allowable limits for all permitted observing conditions. During the first year in orbit, we used the STOP analysis predictions to help interpret the measured performance as a function of environmental variables. Unpredicted disturbances were discovered (heat pulses from instrument operation propagating into the telescope) and unexpected sensitivities were revealed (high dependence of $T_{\rm BP}$ on $\ac{SAA}$ , nearly absent dependence on $\ac{AA}$ after the attitude domain was redefined for straylight avoidance). In-orbit temperature variations are small (<300 mK) and so are their effects on the telescope structure (displacements < 1 $\mu$ m, rotations < 1 $\mu$ rad), but they are detected in the time histories of the image quality metrics and are a non-negligible factor in the $\ac{PSF}$ stability budget demanded by the weak lensing science ( $\Delta e < 2{\times}10^{-3}$ over 11 000 s). Taking everything into account, our analysis confirms the excellent overall performance of the telescope.

</div>

<div id="div_fig1">

<img src="tmp_2512.01075/./HK_timehistories_2023.png" alt="Fig14" width="100%"/>

**Figure 14. -** Time histories of spacecraft attitude and key thermal performance parameters obtained from spacecraft telemetry during the \Euclid performance verification phase in the period from 2023-10-01 00:00:00 UTC until 2023-11-26 00:00:00 UTC. ** Upper panel:** the variations in SAA and AA. Dedicated rotations of the SAA (red line) and AA (blue line) were performed to verify the thermal response of the system. ** Upper middle panel:** the history of the cadence (in cycles per hour) of the NISP grism and filter wheel (GFW, red line), and the VIS shutter (RSU, blue line). ** Lower middle panel:** the history of the VIS radiator temperature registered by sensor THM02 (blue line), which is an indicator of the VIS instrument activity by means of its power dissipation. ** Lower panel:** the temperature responses of a baseplate sensor placed near the VIS bracket (THM05, red line) and a sensor mounted directly on the VIS \ac{FPA} bracket (THM01, blue line). (*fig:temp-history-2023*)

</div>
<div id="div_fig2">

<img src="tmp_2512.01075/./IQ_on-orbit_SepDec2024.png" alt="Fig9" width="100%"/>

**Figure 9. -** Similar as \cref{fig:IQ_history_2023} but for the \ac{IQ} metrics observed in the period shown in \cref{fig:temp-history-2024}, covering 4 months of nominal mission, from 2024-09-01 (mission day 428) until 2024-12-31 (mission day 549). (*fig:IQ_history_2024*)

</div>
<div id="div_fig3">

<img src="tmp_2512.01075/./Tbp_static_cases.png" alt="Fig3" width="100%"/>

**Figure 3. -** Baseplate average temperature versus STOP case number, see \cref{subsub:steady-state-thermal} for the sequencing description. ** Upper panel (a):** full-length \ac{MPS} boom, the topside can be illuminated by the Sun. ** Lower panel (b):** half-length \ac{MPS} boom, the topside cannot be illuminated by the Sun. (*FigTBplot*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2512.01075"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\orcid}[1]$
$\newcommand{\arraystretch}{1.3}$</div>



<div id="title">

# $\Euclid$: the first statistical census of dusty and massive objects in the ERO/Perseus field$\thanks{This paper is published on behalf of the Euclid Consortium.}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2512.01489-b31b1b.svg)](https://arxiv.org/abs/2512.01489)<mark>Appeared on: 2025-12-02</mark> -  _23 pages, 8 figures. Accepted for publication in A\&A_

</div>
<div id="authors">

G. Girardi, et al. -- incl., <mark>E. Bañados</mark>, <mark>S. Belladitta</mark>, <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** Our comprehension of the history of star formation at $z>3$ strongly relies on rest-frame UV observations. However, this selection systematically misses the most dusty and massive sources, resulting in an incomplete census at earlier times. Infrared facilities such as _Spitzer_ and the _James Webb_ Space Telescope have shed light on a hidden population lying at $z=3$ -- $6$ , characterised by extreme red colours, named HIEROs (HST-to-IRAC extremely red objects), identified by the colour criterion $\HE - \mathrm{ch2} > 2.25$ . Recently, $\Euclid$ Early Release Observations (ERO) have opened the possibility to further study such objects, exploiting the comparison between $\Euclid$ and ancillary _Spitzer_ /IRAC observations. The aim of this paper is to conduct an investigation on a small area of $232$ arcmin $^2$ to investigate the effectiveness of this synergy in characterising this population. We utilize catalogues in the Perseus field across the VIS and NISP bands, supplemented by data from the four _Spitzer_ channels and several ground-based MegaCam bands ( _u_ , _g_ , _r_ , ${\rm H} \alpha$ , _i_ , and _z_ ) already included in the ERO catalogue. We select $121$ HIEROs by applying the $\HE - \mathrm{ch2} > 2.25$ colour cut, clean this sample of globular clusters and brown dwarfs, and then inspect by eye the multiband cutouts of each source, ending with $42$ reliable HIEROs. Photometric redshifts and other physical properties of the final sample have been estimated using the SED-fitting software \texttt{Bagpipes} . From the $z_{\mathrm{phot}}$ and $M_*$ values, we compute the galaxy stellar mass function at $3.5<z<5.5$ . When we exclude all galaxies that could host an AGN, or where the stellar masses might be overestimated, we still find that the high-mass end of the galaxy stellar mass function is similar to previous estimates, indicating that the true value could be even higher. This investigation highlights the importance of a deeper study of this still mysterious population, in particular, to assess its contribution to the cosmic star-formation rate density and its agreement with current galaxy evolution and formation models. Indeed, these early results show $\Euclid$ 's capabilities to push the boundaries of our understanding of obscured star formation across a wide range of epochs.

</div>

<div id="div_fig1">

<img src="tmp_2512.01489/./redshift_vs_stellarM_curve_Mlim.png" alt="Fig3.1" width="33%"/><img src="tmp_2512.01489/./redshift_vs_extinction_HIEROs.png" alt="Fig3.2" width="33%"/><img src="tmp_2512.01489/./stellarM_vs_Av_HIEROs.png" alt="Fig3.3" width="33%"/>

**Figure 3. -** Properties of the HIEROs. Panel a) shows the redshift versus stellar mass. The red circles show the most massive candidates, with stellar mass greater than $10^{11.7} M_\odot$. These objects are from here on defined as `overmassive' and shown with the same symbol, i.e., red circles, in all other panels. The solid lines report the minimum observable stellar mass producing an IRAC ch2 magnitude of $22.7$. All three curves represent star-forming galaxies, the green with an age between the redshift epoch and the burst peak of $200$ Myr, the teal one of $100$ Myr, and the dark blue one of $50$ Myr. In panel b) the redshift versus dust attenuation distribution is reported. Panel c) shows the dust attenuation versus stellar mass. The cyan solid line shows the relation from \cite{mclure2018dust}, while the magenta line delimits the area identifying the so-called HELM galaxies \citep{bisigello2025spectroscopic}. Different symbols report values from previous studies, as indicated in the legend. (*fig:physical_param*)

</div>
<div id="div_fig2">

<img src="tmp_2512.01489/./HIEROs_cutouts/combined_cutouts_1_limited_vmin_vmax.png" alt="Fig9.1" width="9%"/><img src="tmp_2512.01489/./HIEROs_cutouts/combined_cutouts_2_limited_vmin_vmax.png" alt="Fig9.2" width="9%"/><img src="tmp_2512.01489/./HIEROs_cutouts/combined_cutouts_3_limited_vmin_vmax.png" alt="Fig9.3" width="9%"/><img src="tmp_2512.01489/./HIEROs_cutouts/combined_cutouts_4_limited_vmin_vmax.png" alt="Fig9.4" width="9%"/><img src="tmp_2512.01489/./HIEROs_cutouts/combined_cutouts_5_limited_vmin_vmax.png" alt="Fig9.5" width="9%"/><img src="tmp_2512.01489/./HIEROs_cutouts/combined_cutouts_6_limited_vmin_vmax.png" alt="Fig9.6" width="9%"/><img src="tmp_2512.01489/./HIEROs_cutouts/combined_cutouts_7_limited_vmin_vmax.png" alt="Fig9.7" width="9%"/><img src="tmp_2512.01489/./HIEROs_cutouts/combined_cutouts_8_limited_vmin_vmax.png" alt="Fig9.8" width="9%"/><img src="tmp_2512.01489/./HIEROs_cutouts/combined_cutouts_9_limited_vmin_vmax.png" alt="Fig9.9" width="9%"/><img src="tmp_2512.01489/./HIEROs_cutouts/combined_cutouts_10_limited_vmin_vmax.png" alt="Fig9.10" width="9%"/><img src="tmp_2512.01489/./HIEROs_cutouts/combined_cutouts_11_limited_vmin_vmax.png" alt="Fig9.11" width="9%"/>

**Figure 9. -** Cutouts of all $42$ sources present in our final sample. ID $=13$ has already been shown in the main text. For IDs $ 1$ to $14$ and ID $=42, 43$, the sources belong to the first pointing, therefore having ch3 and not ch4. From ID $= 15$ until $43$, the objects are observed in ch4 and not in ch3; some of them are missing ch1 data because of the different image sizes (see the explanation in Sect. \ref{photom_analyses}). These cutouts are $15" \times 15"$. (*fig:cutouts*)

</div>
<div id="div_fig3">

<img src="tmp_2512.01489/./bagpipes_plots/Object42.png" alt="Fig2.1" width="33%"/><img src="tmp_2512.01489/./bagpipes_plots/Object28.png" alt="Fig2.2" width="33%"/><img src="tmp_2512.01489/./bagpipes_plots/Object11.png" alt="Fig2.3" width="33%"/>

**Figure 2. -** \texttt{Bagpipes} fits to the SEDs for three representative objects, IDs $42$, $28$, and $11$. The posterior probability distributions for the redshifts, PDF($z$), are shown as insets. Photometric detections are shown by the coloured circles and 3$ \sigma$ upper limits are plotted as triangles. The coloured lines show the three different fits described in the text. The PDF for ID $42$ peaks at low redshift, while ID $28$ is bimodal and ID $11$ has a single high-redshift peak, making it representative of the objects that are likely to be HIEROs. (*fig:SED_fitting_examples*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2512.01489"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# Substellar initial mass function of Trumpler 14

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2512.01606-b31b1b.svg)](https://arxiv.org/abs/2512.01606)<mark>Appeared on: 2025-12-02</mark> - 

</div>
<div id="authors">

T. Rom, et al. -- incl., <mark>M. Schirmer</mark>

</div>
<div id="abstract">

**Abstract:** Young, massive stellar clusters offer a prime setting to explore brown dwarf (BD) formation under high densities and intense UV radiation. Trumpler 14 (Tr 14), a $\sim$ 1 Myr-old cluster located at a distance of 2.4 kpc, and particularly rich in O-type stars, is an ideal target for such a study. Our goal is to measure the initial mass function (IMF) in the young massive, high UV flux cluster. We present the deepest study to date of the IMF in Tr 14 based on  Gemini Multi-Conjugate Adaptive Optics System/ Gemini-South Adaptive Optics Imager imaging. We construct the IMF using both the Besançon Galactic model and an observational control field from VISTA for background correction. Completeness was assessed using artificial-star tests and applied to the IMF derivation. We estimate the IMF down to the 20 \% completeness limit found at $\sim 0.01 M_\odot$ . Using the control field-based IMF as our primary result, we find a slope of $\alpha=0.14\pm0.19$ for masses between 0.01–0.2 $M_\odot$ , and $\alpha=1.72\pm0.04$ for 0.2–4.5 $M_\odot$ , where $dN/dM \propto M^{-\alpha}$ . The low-mass slope is largely influenced by the incompleteness-affected lowest bin; excluding it brings our results into agreement with those from other young clusters. The resulting median for the star-to-BD ratio in the $0.03-1 M_\odot$ mass range is 4.0, with a 95 \% confidence interval of 2.8-5.8. Our analysis reveals that Tr 14 hosts a relatively flat substellar IMF, but this is strongly influenced by the lowest-mass bin, which may be affected by incompleteness. When that bin is excluded, the IMF becomes consistent with those of other regions. The star-to-BD ratio falls within the usually observed $\sim$ 3–6 range, indicating that brown dwarfs with masses above $\sim$ 0.03 $M_\odot$ form with similar efficiency across environments. However, the relative lack of objects below this threshold suggests that the presence of an environment with both high stellar density and FUV flux may play a role in shaping the IMF by suppressing the formation of BDs at masses $<0.03 M_\odot$ .

</div>

<div id="div_fig1">

<img src="tmp_2512.01606/./extinction_test.png" alt="Fig7" width="100%"/>

**Figure 7. -** CMDs for the Besançon model in the upper panels and VISTA CF in the lower panels. The black dots show all the objects in the line of sight of Tr 14 in all panels. The grey dots are the model foreground, while the red ones are the reddened model background. The amount of reddening is given above each corresponding panel. The orange dots represent the reddened CF. Again, the amount of reddening is given above each panel. Each extinction is chosen to decontaminate the most amount of background sources. (*fig:CF*)

</div>
<div id="div_fig2">

<img src="tmp_2512.01606/./plots2.png" alt="Fig6" width="100%"/>

**Figure 6. -** Left and central panel show the CMDs. The mean uncertainty is in the upper right part of the panels. The red solid line represents the 1 Myr isochrone combined from the PARSEC and BT-Settl models at 1 $M_\odot$(\citealt{bressan2012parsec, chen2014improving, baraffe2015new}). The reddening vector of Av=3 mag is shown  {\citep{extinction_law}}. The dashed black line in the first two panels represents the 90\% completeness limit, while the dotted black line represents the 50\% completeness limit and the solid black line the 20\% completeness limit. The right panel represents the  colour- colour diagram (CCD). The solid red line is the isochrone as in the CMDs. The dashed red lines are parallel to extinction vectors, while the red crosses represent Av=0, 5 and 10 mag. The dashed-dotted magenta line represents the T-Tauri stars locus with the corresponding uncertainties as blue dashed lines \citep{meyer+1997}, transformed to 2MASS system using \cite{carpenter_color_2001}. The average uncertainty is in the lower-right corner in the right-most panel, in other panels it is up right. The starting points for the left and middle reddening vector are determined by the masses in the model (<0.5 $M_\odot$ and >=0.005 $M_\odot$, respectively) while the right reddening vector is from \cite{muzic2017}, adapting the locus of Herbig AeBe stars from \cite{Hernandez_2005}. (*fig:CMD_CCD*)

</div>
<div id="div_fig3">

<img src="tmp_2512.01606/./mass_ext_1.png" alt="Fig9" width="100%"/>

**Figure 9. -** Example of mass derivation for one of the sources. The first two panels display the 1 Myr isochrone (red solid line) combined from the PARSEC and BT-Settl models at 1 $M_\odot$ mass (\citealt{bressan2012parsec, chen2014improving, baraffe2015new}) and the source with its error bars (orange solid cross). The grey dots represent the generated magnitude points from 1000 runs. In the first panel a line parallel to the extinction vector would show where the mass lies on the model. In the second panel, the red dotted lines are parallel to the reddening vectors. Red crosses mark the extinction in 5 mag steps from 0 to 15 mag. The third and the fourth panel display the mass and extinction distribution, respectively, obtained from 1000 runs. (*fig:mass_ext*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2512.01606"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

140  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

17  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

9  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
